# Data Loading and Exploration

##### This section loads the breast cancer dataset, explores its structure, and provides initial insights through summary statistics and a pairplot visualization.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer

# Load the breast cancer dataset
def load_data():
    cancer = load_breast_cancer()
    return pd.DataFrame(cancer.data, columns=cancer.feature_names), cancer.target

cancer_df, target = load_data()

# Exploratory Data Analysis
def explore_data(df):
    print("Dataset Shape:", df.shape)
    print("\nSummary Statistics:\n", df.describe())
    sns.pairplot(df.iloc[:, :5])  # Pairplot for first 5 columns to get a feel of the data
    plt.show()

explore_data(cancer_df)


# Data Cleaning

##### This section handles any potential data issues, including missing values, duplicates, outliers, and incorrect data types. Even if the dataset is clean, these steps ensure that it can handle any dirty data

In [ ]:
# Data Cleaning: Handle missing values, duplicates, and outliers
def clean_data(df):
    # 1. Handle Missing Values
    missing_values = df.isnull().sum()
    
    if missing_values.sum() > 0:
        print(f"Missing values found:\n{missing_values}")
        # Strategy to handle missing values: Fill numeric columns with the mean
        df.fillna(df.mean(), inplace=True)
        print("Missing values handled by replacing with mean.")
    else:
        print("No missing values found.")
    
    # 2. Handle Duplicates
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        print(f"Found {duplicates} duplicate rows.")
        df.drop_duplicates(inplace=True)
        print("Duplicates removed.")
    else:
        print("No duplicate rows found.")
    
    # 3. Handle Outliers (Optional Step)
    z_scores = (df - df.mean()) / df.std()
    outliers = (z_scores.abs() > 3).sum()
    
    if outliers.sum() > 0:
        print(f"Outliers detected in the following features:\n{outliers[outliers > 0]}")
        # Capping the outliers at 3 standard deviations for each column individually
        for col in df.columns:
            mean = df[col].mean()
            std = df[col].std()
            lower_bound = mean - 3 * std
            upper_bound = mean + 3 * std
            df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
        print("Outliers capped at 3 standard deviations.")
    else:
        print("No significant outliers found.")
    
    # 4. Ensure Correct Data Types
    for col in df.columns:
        if df[col].dtype == 'object':
            print(f"Converting column {col} to categorical type.")
            df[col] = df[col].astype('category')
        elif df[col].dtype in ['float64', 'int64']:
            df[col] = pd.to_numeric(df[col])
    
    return df

# Clean the cancer dataset
cancer_df_cleaned = clean_data(cancer_df)


# Data Scaling

##### This section scales the features using StandardScaler, ensuring that all features are on the same scale before applying PCA. Scaling is essential for PCA as it is sensitive to the variance of each feature.

In [4]:
from sklearn.preprocessing import StandardScaler

# Standardize the dataset
def scale_data(df):
    scaler = StandardScaler()
    return scaler.fit_transform(df)

X_scaled = scale_data(cancer_df_cleaned)


# PCA Implementation

##### This section applies Principal Component Analysis (PCA) to reduce the dataset to 2 components. It also visualizes the explained variance ratio to show how much information is retained by each component.

In [ ]:
from sklearn.decomposition import PCA

# Apply PCA and visualize explained variance
def apply_pca(X, n_components=2):
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    
    print(f"Explained Variance Ratio: {pca.explained_variance_ratio_}")
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, n_components + 1), pca.explained_variance_ratio_, marker='o', linestyle='--')
    plt.xlabel('Principal Components')
    plt.ylabel('Explained Variance Ratio')
    plt.title('Explained Variance by PCA Components')
    plt.show()
    
    return X_pca

X_pca = apply_pca(X_scaled, n_components=2)


# Visualize the PCA Components

##### After reducing the dataset with PCA, this section visualizes the transformed data using a scatter plot. The plot helps to see how well the two principal components separate the data based on the cancer labels (benign vs malignant).

In [ ]:
# Visualizing PCA Components with Cancer Labels
def visualize_pca(X_pca, target):
    plt.figure(figsize=(8, 6))
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=target, cmap='coolwarm', edgecolor='k', s=60)
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.title('PCA of Breast Cancer Dataset')
    plt.show()

visualize_pca(X_pca, target)


# Logistic Regression with Cross-Validation

##### This section applies Logistic Regression on the PCA-reduced dataset. Cross-validation is used to evaluate the model's performance across multiple splits of the data, ensuring more robust results.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

# Split data and apply logistic regression
def logistic_regression(X_pca, target):
    X_train, X_test, y_train, y_test = train_test_split(X_pca, target, test_size=0.3, random_state=42)
    
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    
    # Cross-validation for robust performance evaluation
    cv_scores = cross_val_score(logreg, X_train, y_train, cv=5)
    print(f"Cross-validation scores: {cv_scores}")
    print(f"Mean Cross-validation Accuracy: {cv_scores.mean()}")
    
    y_pred = logreg.predict(X_test)
    
    return y_test, y_pred

y_test, y_pred = logistic_regression(X_pca, target)


# Evaluate the Model

##### In this section, I evaluate the logistic regression model using metrics like classification report (which includes precision, recall, F1-score) and ROC-AUC score to measure the model's ability to distinguish between classes.

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# Evaluate the performance of logistic regression
def evaluate_model(y_test, y_pred):
    print("Classification Report:\n", classification_report(y_test, y_pred))
    roc_auc = roc_auc_score(y_test, y_pred)
    print(f"ROC-AUC Score: {roc_auc}")

evaluate_model(y_test, y_pred)
